In [ ]:
#libraries
from osgeo import gdal
import numpy as np

In [ ]:
# opening the list of swath array shapes
swath_array_shapes = []

In [ ]:
#looping through the swaths
for i in range(3):
    for j in range(9):
        
        # reading the i_j swath
        dataset = (gdal.Open(
            r'tif_files/S1A_IW_SLC__1SDV_20210705T090737_'
            + '20210705T090804_038640_048F41_253E_split_'
            + str(i + 1) + '_' + str(j + 1)
            + '_Orb_TNR_Cal_deb_ML_Lee3.tif'))
        
        #getting the raster bands
        band1 = dataset.GetRasterBand(1) # Red channel
        band2 = dataset.GetRasterBand(2) # Green channel
        b1 = band1.ReadAsArray()
        b2 = band2.ReadAsArray()
        b3 = np.divide(b1, b2) # Blue channel
        b3 = np.nan_to_num(b3)
        swath_array = np.dstack((b1, b2, b3))

        #remove black boards
        while np.prod(sum(swath_array)) == 0:
            if sum(sum(swath_array[:,0,:])) == 0:
                swath_array = np.delete(swath_array, 0, 1)
            if sum(sum(swath_array[:,-1,:])) == 0:
                swath_array = np.delete(swath_array, -1, 1)
            if sum(sum(swath_array[0,:,:])) == 0:
                swath_array = np.delete(swath_array, 0, 0)
            if sum(sum(swath_array[-1,:,:])) == 0:
                swath_array = np.delete(swath_array, -1, 0)

        #flip array horizontally
        swath_array = np.flip(swath_array, 1)
        
        #filling the swath array shapes list  
        swath_array_shapes.append(swath_array.shape)
        
        #saving 3D-arrays as 2D-txt
        reshaped_array = swath_array.reshape(swath_array.shape[0], -1)
        np.savetxt("txt_files/swath_" + str(i + 1) + "_" + str(j + 1)
                   + ".txt", reshaped_array)
        
        #dimensions of cropped samples in each swath
        x_len = 358
        y_len = 363
        
        #number of crooped samples for each row in each swath
        if i == 0:
            samples = 19
        elif i == 1:
            samples = 17
        else:
            samples = 16
            
        #looping through samples in a given swath
        for k in range(4):
            for l in range(samples):
                cropped_array = swath_array[
                    0 + y_len * k : y_len * (1 + k),
                    0 + x_len * l : x_len * (1 + l), :]
                reshaped_array = cropped_array.reshape(
                    cropped_array.shape[0], -1)
                (np.savetxt("txt_files/cropped_" + str(i + 1) + "_"
                            + str(j + 1) + "_" + str(k + 1) + "_" +
                            str(l + 1) + ".txt", reshaped_array))

In [ ]:
#saving the swath array shapes list as txt
np.savetxt("txt_files/swath_array_shapes.txt", swath_array_shapes)